In [1]:
import gct

In [5]:
prefix = "K_DoesAlgSupportDirTest"

def setup_data():
    gct.remove_data(prefix + '_*', dry_run=False)
    data1 = gct.random_dataset.generate_directed_unweighted_random_graph_LFR(name=prefix + "_1", \
                                   N=128, k=16, maxk=32, mu=0.2, minc=32)
    data2 = data1.as_undirected(newname=prefix + "_2")
    return data1,data2


In [6]:
data_w,data_uw=setup_data()

2020-04-23 00:52:52,964 - RandomGenerator:LFR - INFO - Runing '/opt/gct/submodules/CommunityDetectionCodes/lfr_dir_net -minc 32 -mu 0.2 -maxk 32 -k 16 -N 128'  with seed 768120
INFO:RandomGenerator:LFR:Runing '/opt/gct/submodules/CommunityDetectionCodes/lfr_dir_net -minc 32 -mu 0.2 -maxk 32 -k 16 -N 128'  with seed 768120
2020-04-23 00:52:52,965 - RandomGenerator:LFR - INFO - working dir: /tmp/tmpj4b7l8f9
INFO:RandomGenerator:LFR:working dir: /tmp/tmpj4b7l8f9
2020-04-23 00:52:53,002 - RandomGenerator:LFR - INFO - min node is 1. will make it 0
INFO:RandomGenerator:LFR:min node is 1. will make it 0
2020-04-23 00:52:53,011 - Dataset:K_DoesAlgSupportDirTest_1 - INFO - writing/data/data/K_DoesAlgSupportDirTest_1/edges.parq
INFO:Dataset:K_DoesAlgSupportDirTest_1:writing/data/data/K_DoesAlgSupportDirTest_1/edges.parq
2020-04-23 00:52:53,372 - Clustering - INFO - Writing /data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
INFO:Clustering:Writing /data/data/K_DoesAlgSupportDirTest_1/gt_default

In [7]:
data_w.name,data_uw.name

('K_DoesAlgSupportDirTest_1', 'K_DoesAlgSupportDirTest_2')

In [8]:
!pip install func-timeout

You are using pip version 9.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
import os
from func_timeout import func_timeout, FunctionTimedOut
from gct import config,utils
def has_run_1(runame, dsname):
    fpath = os.path.join(config.get_result_file_path(dsname=dsname, runname=runame), 'result.txt')
    return utils.file_exists(fpath)

def has_run(runame, dsname):
    fpath = os.path.join("/{}/tmp/{}_{}.done".format('data', runame,dsname))
    return utils.file_exists(fpath)

def timeout(func, args=(), kwargs={}, timeout_duration=10, default=None):
    result=default
    try:
        result = func_timeout(timeout_duration, func, args=args, kwargs=kwargs)
    except FunctionTimedOut:
        print("timeout", func,args,kwargs)
    except:
        print ("Unexpected error:", sys.exc_info()[0])
    return result

def timeout0(func, args=(), kwargs={}, timeout_duration=10, default=None):
    import signal

    class TimeoutError(Exception):
        pass

    def handler(signum, frame):
        raise TimeoutError()

    # set the timeout handler
    signal.signal(signal.SIGALRM, handler) 
    signal.alarm(timeout_duration)
    try:
        result = func(*args, **kwargs)
    except TimeoutError as exc:
        result = default
    finally:
        signal.alarm(0)

    return result


In [10]:

def run_alg(alg,dsname, check_skip=True):
    try:
        runname = alg
        if not check_skip or (check_skip and not has_run(runname, dsname)):
            print ("runing ", alg, dsname)
            if alg =='cdc_SVINET':
                gct.run_alg(runname=runname, data=gct.load_local_graph(dsname), algname=alg, seed=123, max_iterations=1000)        
            else:
                gct.run_alg(runname=runname, data=gct.load_local_graph(dsname), algname=alg, seed=123)
            print ("finished", alg,dsname)
        else:
            print("skip", runname,dsname)
    except:
        print("except", runname,dsname)
    return True
    

def run_alg_timeout(alg,dsname,timeout_duration=600):
    if not timeout(run_alg, (alg,dsname), timeout_duration=timeout_duration):
        print("tiemout", alg,dsname)
    fpath = os.path.join("/{}/tmp/{}_{}.done".format('data', alg,dsname))
    !touch {fpath}

        


In [11]:
run_alg("oslom_lpm", data_w.name)

2020-04-23 00:53:02,581 - Dataset:K_DoesAlgSupportDirTest_1 - INFO - writing edges to /data/data/K_DoesAlgSupportDirTest_1/edges.txt
INFO:Dataset:K_DoesAlgSupportDirTest_1:writing edges to /data/data/K_DoesAlgSupportDirTest_1/edges.txt
2020-04-23 00:53:02,582 - Dataset:K_DoesAlgSupportDirTest_1 - INFO - reading /data/data/K_DoesAlgSupportDirTest_1/edges.parq
INFO:Dataset:K_DoesAlgSupportDirTest_1:reading /data/data/K_DoesAlgSupportDirTest_1/edges.parq


runing  oslom_lpm K_DoesAlgSupportDirTest_1


2020-04-23 00:53:02,921 - Dataset:K_DoesAlgSupportDirTest_1 - INFO - finish writing /data/data/K_DoesAlgSupportDirTest_1/edges.txt
INFO:Dataset:K_DoesAlgSupportDirTest_1:finish writing /data/data/K_DoesAlgSupportDirTest_1/edges.txt
2020-04-23 00:53:02,922 - oslom_lpm - INFO - Running /opt/gct/submodules/OSLOM/lpm -f /tmp/tmpsyhg9b05/edges.txt -r 1 -seed 123
INFO:oslom_lpm:Running /opt/gct/submodules/OSLOM/lpm -f /tmp/tmpsyhg9b05/edges.txt -r 1 -seed 123
2020-04-23 00:53:02,933 - oslom_lpm - INFO - Made 4 clusters in 0.009885 seconds
INFO:oslom_lpm:Made 4 clusters in 0.009885 seconds


finished oslom_lpm K_DoesAlgSupportDirTest_1


True

In [13]:
run_alg_timeout("oslom_lpm", data_w.name)

skip oslom_lpm K_DoesAlgSupportDirTest_1


In [14]:
datasets=[data_w.name,data_uw.name]
for alg in gct.list_algorithms():
    for name in datasets:
        run_alg_timeout(alg, name)

2020-04-23 00:53:09,269 - oslom_Infohiermap - INFO - Running /opt/gct/submodules/OSLOM/infohiermap_dir 123 /tmp/tmpn8zm3xhy/pajek.txt 1
INFO:oslom_Infohiermap:Running /opt/gct/submodules/OSLOM/infohiermap_dir 123 /tmp/tmpn8zm3xhy/pajek.txt 1
2020-04-23 00:53:09,290 - oslom_Infohiermap - INFO - Made 6 clusters in 0.011665 seconds
INFO:oslom_Infohiermap:Made 6 clusters in 0.011665 seconds


runing  oslom_Infohiermap K_DoesAlgSupportDirTest_1
finished oslom_Infohiermap K_DoesAlgSupportDirTest_1


2020-04-23 00:53:09,401 - Dataset:K_DoesAlgSupportDirTest_2 - INFO - writing edges to /data/data/K_DoesAlgSupportDirTest_2/edges.txt
INFO:Dataset:K_DoesAlgSupportDirTest_2:writing edges to /data/data/K_DoesAlgSupportDirTest_2/edges.txt
2020-04-23 00:53:09,403 - Dataset:K_DoesAlgSupportDirTest_2 - INFO - reading /data/data/K_DoesAlgSupportDirTest_2/edges.parq
INFO:Dataset:K_DoesAlgSupportDirTest_2:reading /data/data/K_DoesAlgSupportDirTest_2/edges.parq


runing  oslom_Infohiermap K_DoesAlgSupportDirTest_2


2020-04-23 00:53:09,592 - Dataset:K_DoesAlgSupportDirTest_2 - INFO - finish writing /data/data/K_DoesAlgSupportDirTest_2/edges.txt
INFO:Dataset:K_DoesAlgSupportDirTest_2:finish writing /data/data/K_DoesAlgSupportDirTest_2/edges.txt
2020-04-23 00:53:09,610 - oslom_Infohiermap - INFO - Running /opt/gct/submodules/OSLOM/infohiermap_undir 123 /tmp/tmps7ftwdh8/pajek.txt 1
INFO:oslom_Infohiermap:Running /opt/gct/submodules/OSLOM/infohiermap_undir 123 /tmp/tmps7ftwdh8/pajek.txt 1
2020-04-23 00:53:09,636 - oslom_Infohiermap - INFO - Made 6 clusters in 0.016581 seconds
INFO:oslom_Infohiermap:Made 6 clusters in 0.016581 seconds


...

runing  alg_lso_cluster K_DoesAlgSupportDirTest_2
finished alg_lso_cluster K_DoesAlgSupportDirTest_2


2020-04-23 01:24:54,735 - alg_streamcom - INFO - Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgSupportDirTest_1/edges.txt -o output /data/data/K_DoesAlgSupportDirTest_1/edges.txt
INFO:alg_streamcom:Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgSupportDirTest_1/edges.txt -o output /data/data/K_DoesAlgSupportDirTest_1/edges.txt
2020-04-23 01:24:54,746 - alg_streamcom - INFO - Made 68 clusters in 0.009565 seconds
INFO:alg_streamcom:Made 68 clusters in 0.009565 seconds


runing  alg_streamcom K_DoesAlgSupportDirTest_1
finished alg_streamcom K_DoesAlgSupportDirTest_1


2020-04-23 01:24:54,862 - alg_streamcom - INFO - Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgSupportDirTest_2/edges.txt -o output /data/data/K_DoesAlgSupportDirTest_2/edges.txt
INFO:alg_streamcom:Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgSupportDirTest_2/edges.txt -o output /data/data/K_DoesAlgSupportDirTest_2/edges.txt
2020-04-23 01:24:54,874 - alg_streamcom - INFO - Made 72 clusters in 0.009777 seconds
INFO:alg_streamcom:Made 72 clusters in 0.009777 seconds


runing  alg_streamcom K_DoesAlgSupportDirTest_2
finished alg_streamcom K_DoesAlgSupportDirTest_2


In [15]:
results = {}
d = gct.list_all_clustering_results(print_format=False)
for u in d:
    if prefix + "_" in u:
        for run in d[u]:
            a = gct.load_clustering_result(u, run)
            print (u, run)
            results[u + run] = a



K_DoesAlgSupportDirTest_1 oslom_lpm
K_DoesAlgSupportDirTest_1 oslom_Infohiermap
K_DoesAlgSupportDirTest_1 oslom_Infomap
K_DoesAlgSupportDirTest_1 oslom_OSLOM
K_DoesAlgSupportDirTest_1 oslom_copra
K_DoesAlgSupportDirTest_1 oslom_louvain_method
K_DoesAlgSupportDirTest_1 oslom_modopt
K_DoesAlgSupportDirTest_1 pycabem_GANXiSw
K_DoesAlgSupportDirTest_1 pycabem_HiReCS
K_DoesAlgSupportDirTest_1 pycabem_LabelRank
K_DoesAlgSupportDirTest_1 cdc_CONGA
K_DoesAlgSupportDirTest_1 cdc_Connected_Iterative_Scan
K_DoesAlgSupportDirTest_1 cdc_EAGLE
K_DoesAlgSupportDirTest_1 cdc_FastCpm
K_DoesAlgSupportDirTest_1 cdc_GCE
K_DoesAlgSupportDirTest_1 cdc_LinkCommunities
K_DoesAlgSupportDirTest_1 cdc_MOSES
K_DoesAlgSupportDirTest_1 cdc_MSCD_AFG
K_DoesAlgSupportDirTest_1 cdc_MSCD_HSLSW
K_DoesAlgSupportDirTest_1 cdc_MSCD_LFK
K_DoesAlgSupportDirTest_1 cdc_MSCD_LFK2
K_DoesAlgSupportDirTest_1 cdc_MSCD_RB
K_DoesAlgSupportDirTest_1 cdc_MSCD_RN
K_DoesAlgSupportDirTest_1 cdc_MSCD_SO
K_DoesAlgSupportDirTest_1 cdc_MSCD_SO

In [16]:
import pandas as pd
lst = [];
for alg in  gct.list_algorithms():
    for dsname in datasets:
        if dsname.startswith(prefix + '_1'):
            a = [];columns = []
            dsname2 = dsname.replace(prefix + "_1", prefix + "_2")
            a += [alg, dsname, dsname2]
            columns += ['alg', 'data1', 'data2']
            if dsname + alg in results and dsname2 + alg in results:
                cluster1 = gct.to_cluster(results[dsname + alg])
                cluster2 = gct.to_cluster(results[dsname2 + alg])
                compa = gct.ClusterComparator(cluster1, cluster2)
                a += [compa.sklean_nmi(), cluster1.is_overlap, cluster2.is_overlap,
                      cluster1.num_cluster, cluster2.num_cluster,
                     compa.OvpNMI()['NMImax']]
                columns += ['nmi_12', 'ovp1', 'ovp2', "#c1", "#c2", 'ovpnmi_12']

                gt = list(gct.load_local_graph(dsname).get_ground_truth().values())[0]            
                compa = gct.ClusterComparator(gt, cluster1)            
                a += [compa.sklean_nmi(), compa.OvpNMI()['NMImax']]
                columns += ['nmi_t1', 'ovpnmi_t1']

                compa = gct.ClusterComparator(gt, cluster2)            
                a += [compa.sklean_nmi(), compa.OvpNMI()['NMImax']]
                columns += ['nmi_t2', 'ovpnmi_t2']

                lst.append(a)



2020-04-23 01:24:55,055 - ClusterComparator - INFO - resulting 128 nodes out of 129,128
INFO:ClusterComparator:resulting 128 nodes out of 129,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 01:24:55,066 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpy_dl9cgw/cluster1.cnl /tmp/tmpy_dl9cgw/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpy_dl9cgw/cluster1.cnl /tmp/tmpy_dl9cgw/cluster2.cnl > ovpnmioutput
2020-04-23 01:24:55,079 - ClusterComparator - ERROR - Run command with error status code 134
ERROR:ClusterComparator:Run command with error status code 134
2020-04-23 01:24:55,081 - Clustering - INFO - reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
INFO:Clusteri

INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmppol3qrp6/cluster1.cnl /tmp/tmppol3qrp6/cluster2.cnl > ovpnmioutput
2020-04-23 01:24:55,538 - Clustering - INFO - reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
2020-04-23 01:24:55,650 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 01:24:55,659 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp3m6an4z8/cluster1.cnl /tmp/tmp3m6an4z8/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp3m6an4z8/cluster1.cnl 

INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 01:24:56,016 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp3szoug2z/cluster1.cnl /tmp/tmp3szoug2z/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp3szoug2z/cluster1.cnl /tmp/tmp3szoug2z/cluster2.cnl > ovpnmioutput
2020-04-23 01:24:56,034 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use a

2020-04-23 01:24:56,888 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpo1f_ti2u/cluster1.cnl /tmp/tmpo1f_ti2u/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpo1f_ti2u/cluster1.cnl /tmp/tmpo1f_ti2u/cluster2.cnl > ovpnmioutput
2020-04-23 01:24:56,901 - Clustering - INFO - reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
2020-04-23 01:24:56,994 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 01:24:57,002 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi 

INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpsn7o4uy5/cluster1.cnl /tmp/tmpsn7o4uy5/cluster2.cnl > ovpnmioutput
2020-04-23 01:24:58,167 - Clustering - INFO - reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
2020-04-23 01:24:58,174 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 01:24:58,182 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp7ik2reij/cluster1.cnl /tmp/tmp7ik2reij/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp7ik2reij/cluster1.cnl 

...

2020-04-23 01:25:01,426 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp43leyhbm/cluster1.cnl /tmp/tmp43leyhbm/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp43leyhbm/cluster1.cnl /tmp/tmp43leyhbm/cluster2.cnl > ovpnmioutput
2020-04-23 01:25:01,444 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 01:25:01,454 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpqou7ur36/cluster1.cnl /tmp/tmpqou7ur36/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpqou7ur36/cluster1.cnl /tmp/tmpqou7ur

INFO:Clustering:reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
2020-04-23 01:25:01,703 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 01:25:01,711 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpn2qp0tqx/cluster1.cnl /tmp/tmpn2qp0tqx/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpn2qp0tqx/cluster1.cnl /tmp/tmpn2qp0tqx/cluster2.cnl > ovpnmioutput
2020-04-23 01:25:01,723 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/s

INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmppiv_kro1/cluster1.cnl /tmp/tmppiv_kro1/cluster2.cnl > ovpnmioutput
2020-04-23 01:25:01,992 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 01:25:02,002 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpyo2bloa1/cluster1.cnl /tmp/tmpyo2bloa1/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpyo2bloa1/cluster1.cnl /tmp/tmpyo2bloa1/cluster2.cnl > ovpnmioutput
2020-04-23 01:25:02,014 - Clustering - INFO - reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
INFO:Clustering:reading/data/da

INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpslg42dll/cluster1.cnl /tmp/tmpslg42dll/cluster2.cnl > ovpnmioutput
2020-04-23 01:25:02,256 - ClusterComparator - INFO - resulting 126 nodes out of 126,127
INFO:ClusterComparator:resulting 126 nodes out of 126,127
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 01:25:02,267 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp0c3tmv_c/cluster1.cnl /tmp/tmp0c3tmv_c/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp0c3tmv_c/cluster1.cnl /tmp/tmp0c3tmv_c/cluster2.cnl > ovpnmioutput
2020-04-23 01:25:02,281 - Clustering - INFO - reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
INFO:Clustering:reading/data/da

INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-23 01:25:02,557 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp0vowsi2c/cluster1.cnl /tmp/tmp0vowsi2c/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp0vowsi2c/cluster1.cnl /tmp/tmp0vowsi2c/cluster2.cnl > ovpnmioutput
2020-04-23 01:25:02,570 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use a

In [17]:
rdf = pd.DataFrame(lst, columns=columns)
for alg in  gct.list_algorithms():
    if alg not in rdf['alg'].values:
        lst.append([alg] + [None] * (len(columns) - 1))
rdf = pd.DataFrame(lst, columns=columns)    
with pd.option_context('display.max_rows', 2000, 'display.max_columns', 200):
    display (rdf)


,alg,data1,data2,nmi_12,ovp1,ovp2,#c1,#c2,ovpnmi_12,nmi_t1,ovpnmi_t1,nmi_t2,ovpnmi_t2
0,oslom_Infohiermap,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,0.075965,False,False,6.0,6.0,NaN,0.075965,NaN,1.000000,1.000000
1,oslom_Infomap,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,0.919378,False,False,6.0,5.0,0.815637,1.000000,1.000000,0.919378,0.815637
2,oslom_OSLOM,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,1.000000,False,False,6.0,6.0,1.000000,1.000000,1.000000,1.000000,1.000000
3,oslom_copra,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,0.761741,True,True,6.0,5.0,0.719643,0.953598,0.903898,0.843911,0.740825
4,oslom_louvain_method,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,1.000000,False,False,6.0,6.0,1.000000,1.000000,1.000000,1.000000,1.000000
5,oslom_lpm,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,1.000000,False,False,4.0,4.0,1.000000,0.885272,0.714824,0.885272,0.714824
6,oslom_modopt,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,1.000000,False,False,6.0,6.0,1.000000,1.000000,1.000000,1.000000,1.000000
7,pycabem_GANXiSw,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,0.629455,False,False,6.0,2.0,0.303508,1.000000,1.000000,0.629455,0.303508
8,pycabem_HiReCS,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,0.906737,True,True,6.0,5.0,0.796104,0.074949,NaN,0.069614,NaN
9,pycabem_LabelRank,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,1.000000,False,False,6.0,6.0,1.000000,1.000000,1.000000,1.000000,1.000000


In [21]:
run_alg("cdc_CliquePercolation", "K_DoesAlgSupportDirTest_1", check_skip=False)

runing  cdc_CliquePercolation K_DoesAlgSupportDirTest_1
finished cdc_CliquePercolation K_DoesAlgSupportDirTest_1


Error: only undirected and unmirrored graph is supported for CliquePercolation


True